In [298]:
# install stuff, set scientific variables to 999 to avoid scientific notation
if (!require("pacman")) install.packages("pacman")
pacman::p_load(IRdisplay, WDI, tidyr, car, lmtest, psych, robustbase, MASS, dplyr, knitr, broom)


In [299]:
options(scipen = 999)


In [300]:
# get all of the WDI indicators if indicator file is not already downloaded
indicators <- if (file.exists("indicators.csv")) {
    read.csv("indicators.csv")
} else {
    WDI(indicator = c(
        "SL.EMP.TOTL.SP.ZS",
        "SE.XPD.TOTL.GD.ZS",
        "NY.GDP.PCAP.CD",
        "SM.POP.NETM",
        "SM.POP.TOTL.ZS",
        "SE.TER.CUAT.BA.ZS",
        "SP.POP.TOTL",
        "IP.PAT.RESD",
        "BX.TRF.PWKR.CD.DT"
    ), country = "all") %>%
        write.csv("indicators.csv")
}
# rename the columns
wdi_indicators <- rename(
    indicators,
    c(
        "Country Code" = "iso3c",
        "Total Employment" = "SL.EMP.TOTL.SP.ZS",
        "Education Expenditure % of GDP" = "SE.XPD.TOTL.GD.ZS",
        "GDP per capita" = "NY.GDP.PCAP.CD",
        "Net Migration" = "SM.POP.NETM",
        "PCT Tertiary Education" = "SE.TER.CUAT.BA.ZS",
        "Total Population" = "SP.POP.TOTL",
        "PatentsByResidents" = "IP.PAT.RESD",
        "Total Migrant Population" = "SM.POP.TOTL.ZS",
        "Remittances" = "BX.TRF.PWKR.CD.DT"
    )
)


In [301]:
gii <- read.csv("./gii_analysis/gii_2013_2020.csv")
# filter out the rows where the indicator is "Global Innovation Index" and the subindicator type is "Score"
gii_score <- gii %>%
    filter(
        Indicator == "Global Innovation Index",
        Subindicator.Type == "Score (0-100)"
    )
gii_score <- gii_score %>%
    gather(year, value, X2013:X2020) %>%
    select(-Indicator, -Indicator.Id, -Country.Name) %>%
    spread(Subindicator.Type, value)
gii_score <- rename(gii_score, c("Score" = "Score (0-100)"))

# filter out the rows where the indicator is "Global Innovation Index" and the subindicator type is "Rank"
gii_rank <- gii %>%
    filter(Indicator == "Global Innovation Index", Subindicator.Type == "Rank")
gii_rank <- gii_rank %>%
    gather(year, value, X2013:X2020) %>%
    select(-Indicator, -Country.Name, -Indicator.Id) %>%
    spread(Subindicator.Type, value)

# merge into one dataframe
gii_rank_score <- merge(gii_score, gii_rank, by = c("Country.ISO3", "year"))
# remove the X from the year column and convert to integer
gii_rank_score$year <- as.integer(gsub("X", "", gii_rank_score$year))


In [302]:
# merge gii_rank_score with wdi_indicators
gii_wdi <- gii_rank_score %>% right_join(wdi_indicators, by = c("Country.ISO3" = "Country Code", "year" = "year"))
# rename the columns
gii_wdi <- gii_wdi %>%
    rename(c(
        "CountryCode" = "Country.ISO3", "Year" = "year", "Edu" = "PCT Tertiary Education",
        "Mig" = "Net Migration", "EduExp" = "Education Expenditure % of GDP", "TotEmp" = "Total Employment",
        "TotPop" = "Total Population", "Pat" = "PatentsByResidents", "GdpPerCap" = "GDP per capita", "MigPop" = "Total Migrant Population",
        "RemRecv" = "Remittances"
    )) %>%
    select("Year", "CountryCode", "Score", "Rank", "Edu", "Mig", "EduExp", "TotEmp", "TotPop", "Pat", "GdpPerCap", "MigPop", "RemRecv") %>%
    arrange(CountryCode, Year) %>%
    filter(!(CountryCode == ""))

# write out gii_wdi to csv if it doesn't already exist
if (!file.exists("./gii_analysis/gii_wdi.csv")) write.csv(gii_wdi, "./gii_analysis/gii_wdi.csv")
# describe(gii_wdi)
# lag everything.
gii_wdi <- gii_wdi %>%
    group_by(CountryCode) %>%
    mutate(lag.Mig01 = lag(Mig, n = 1, default = NA)) %>%
    mutate(lag.Mig05 = lag(Mig, n = 5, default = NA)) %>%
    mutate(lag.Mig10 = lag(Mig, n = 10, default = NA)) %>%
    mutate(lag.Mig20 = lag(Mig, n = 20, default = NA)) %>%
    mutate(lag.Edu01 = lag(Edu, n = 1, default = NA)) %>%
    mutate(lag.Edu05 = lag(Edu, n = 5, default = NA)) %>%
    mutate(lag.Edu10 = lag(Edu, n = 10, default = NA)) %>%
    mutate(lag.Edu20 = lag(Edu, n = 20, default = NA)) %>%
    mutate(lag.EduExp01 = lag(EduExp, n = 1, default = NA)) %>%
    mutate(lag.EduExp05 = lag(EduExp, n = 5, default = NA)) %>%
    mutate(lag.EduExp10 = lag(EduExp, n = 10, default = NA)) %>%
    mutate(lag.EduExp20 = lag(EduExp, n = 20, default = NA)) %>%
    mutate(lag.TotEmp01 = lag(TotEmp, n = 1, default = NA)) %>%
    mutate(lag.TotEmp05 = lag(TotEmp, n = 5, default = NA)) %>%
    mutate(lag.TotEmp10 = lag(TotEmp, n = 10, default = NA)) %>%
    mutate(lag.TotEmp20 = lag(TotEmp, n = 20, default = NA)) %>%
    mutate(lag.TotPop01 = lag(TotPop, n = 1, default = NA)) %>%
    mutate(lag.TotPop05 = lag(TotPop, n = 5, default = NA)) %>%
    mutate(lag.TotPop10 = lag(TotPop, n = 10, default = NA)) %>%
    mutate(lag.TotPop20 = lag(TotPop, n = 20, default = NA)) %>%
    mutate(lag.MigPop01 = lag(MigPop, n = 1, default = NA)) %>%
    mutate(lag.MigPop05 = lag(MigPop, n = 5, default = NA)) %>%
    mutate(lag.MigPop10 = lag(MigPop, n = 10, default = NA)) %>%
    mutate(lag.MigPop20 = lag(MigPop, n = 20, default = NA))

# head(gii_wdi)

# write out gii_wdi_lagged to csv if it doesn't already exist
if (!file.exists("./gii_analysis/gii_wdi_lagged.csv")) write.csv(gii_wdi, "./gii_analysis/gii_wdi_lagged.csv")


In [303]:
# lag1_Edu_EduExp_Mig_TotEmp <- lm(Score ~ lag.Mig01 + lag.Edu01 + lag.TotEmp01 + lag.EduExp01, data = gii_wdi)
# vif(lag1_Edu_EduExp_Mig_TotEmp)
# bptest(lag1_Edu_EduExp_Mig_TotEmp)
# nobs(lag1_Edu_EduExp_Mig_TotEmp)
# summary(lag1_Edu_EduExp_Mig_TotEmp)

# lag5_Edu_EduExp_Mig_TotEmp <- rlm(Score ~ lag.Mig05 + lag.Edu05 + lag.TotEmp05 + lag.EduExp05, data = gii_wdi)
# vif(lag5_Edu_EduExp_Mig_TotEmp)
# bptest(lag5_Edu_EduExp_Mig_TotEmp)
# nobs(lag5_Edu_EduExp_Mig_TotEmp)
# summary(lag5_Edu_EduExp_Mig_TotEmp)

# model4 <- rlm(Score ~ Mig + Edu + Mig:Edu, data = gii_wdi)
# nobs(model4)
# summary(model4)

# model5 <- lm(Score ~ Mig + EduExp + Mig:EduExp, data = gii_wdi)
# nobs(model5)
# summary(model5)

# lag1_Edu_EduExp_Mig <- rlm(Score ~ lag.Edu01 + lag.EduExp01 + lag.Mig01, data = gii_wdi)
# vif(lag1_Edu_EduExp_Mig)
# bptest(lag1_Edu_EduExp_Mig)
# nobs(lag1_Edu_EduExp_Mig)
# summary(lag1_Edu_EduExp_Mig)

# lag5_Edu_EduExp_Mig <- rlm(Score ~ lag.Edu05 + lag.EduExp05 + lag.Mig05, data = gii_wdi)
# vif(lag5_Edu_EduExp_Mig)
# Anova(lag5_Edu_EduExp_Mig)
# bptest(lag5_Edu_EduExp_Mig)
# nobs(lag5_Edu_EduExp_Mig)
# summary(lag5_Edu_EduExp_Mig)

# There is a ton of heteroskedasticity so we have to log the dependent variable in an atttempt to defeat such

# # log score
# log_lag5_Edu_EduExp_Mig <- rlm(log(Pat) ~ Pat + Mig, data = gii_wdi)
# nobs(log_lag5_Edu_EduExp_Mig)
# Anova(log_lag5_Edu_EduExp_Mig)
# car::vif(log_lag5_Edu_EduExp_Mig)
# lmtest::bptest(log_lag5_Edu_EduExp_Mig)
# summary(log_lag5_Edu_EduExp_Mig)
# coeftest(log_lag5_Edu_EduExp_Mig)
# plot(log_lag5_Edu_EduExp_Mig)
# pat_lagMig <- rlm(log(Score) ~ Edu + lag.Mig20, data = gii_wdi)
# nobs(pat_lagMig)
# vif(pat_lagMig)
# Anova(pat_lagMig)
# coeftest(pat_lagMig)


In [319]:
generate_summaries <- function(linear_model) {
    linear_model %>% display()
    nobs(linear_model) %>% display()
    vif(linear_model) %>% display()
    Anova(linear_model) %>% display()
    bptest(linear_model) %>% display()
    coeftest(linear_model) %>% display()
}

In [320]:
# Score goes up if migration goes up
lagMig01 <- rlm((Score) ~  (EduExp) + (lag.Mig01)  + (TotEmp) + RemRecv + MigPop + log(TotPop), data = gii_wdi)
lagMig05 <- rlm((Score) ~  (EduExp) + (lag.Mig05) + (MigPop) + (TotEmp) + log(TotPop), data = gii_wdi)
lagMig10 <- rlm((Score) ~  (EduExp) + (lag.Mig10) + (MigPop) + (TotEmp) + log(TotPop), data = gii_wdi)
lagMig20 <- rlm((Score) ~  (EduExp) + (lag.Mig20)+ (TotEmp) + MigPop + log(TotPop), data = gii_wdi)
# patents should go up if net migration is positive
pat01 <- rlm(log(Pat) ~ (EduExp) + (lag.Mig01)+ (TotEmp) + MigPop, data = gii_wdi)
pat05 <- rlm(log(Pat) ~ (EduExp) + (lag.Mig05)+ (TotEmp) + MigPop, data = gii_wdi)
pat10 <- rlm(log(Pat) ~ (EduExp) + (lag.Mig10)+ (TotEmp) + MigPop, data = gii_wdi)
pat20 <- rlm(log(Pat) ~ (EduExp) + (lag.Mig20)+ (TotEmp) + MigPop, data = gii_wdi)
# remittances recieved (if net migration is negative, then remittances recieved should be positive)
remRecv01 <- rlm(log1p(RemRecv) ~ (EduExp) + (lag.Mig01)+ (TotEmp) + MigPop, data = gii_wdi)
remRecv05 <- rlm(log1p(RemRecv) ~ (EduExp) + (lag.Mig05)+ (TotEmp) + MigPop, data = gii_wdi)
remRecv10 <- rlm(log1p(RemRecv) ~ (EduExp) + (lag.Mig10)+ (TotEmp) + MigPop, data = gii_wdi)
remRecv20 <- rlm(log1p(RemRecv) ~ (EduExp) + (lag.Mig20)+ (TotEmp) + MigPop, data = gii_wdi)

In [321]:
generate_summaries(lagMig01)

Call:
rlm(formula = (Score) ~ (EduExp) + (lag.Mig01) + (TotEmp) + RemRecv + 
    MigPop + log(TotPop), data = gii_wdi)
Converged in 8 iterations

Coefficients:
        (Intercept)              EduExp           lag.Mig01              TotEmp 
17.9520214775489144  1.5038062119858091  0.0000090417759835  0.0089736143070059 
            RemRecv              MigPop         log(TotPop) 
 0.0000000002177717  0.7360422910912170  0.3243499837475142 

Degrees of freedom: 129 total; 122 residual
  (16053 observations deleted due to missingness)
Scale estimate: 9.09 

[1] 129

EduExp   lag.Mig01      TotEmp     RemRecv      MigPop log(TotPop) 
   1.086953    1.169068    1.099475    1.748326    1.138103    1.838667

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,7.42484871,0.007378865565310205842
lag.Mig01,1,6.64400485,0.011138657287306541130
TotEmp,1,0.01106904,0.916382362520412607765
RemRecv,1,2.75070886,0.099780217070986818029
MigPop,1,81.57238214,0.000000000000003072316
log(TotPop),1,0.23168200,0.631141648243460862133
Residuals,122,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 40.427, df = 6, p-value = 0.0000003754



z test of coefficients:

                     Estimate        Std. Error z value              Pr(>|z|)
(Intercept) 17.95202147754891 11.65751846507363  1.5400              0.123572
EduExp       1.50380621198581  0.55188433890384  2.7249              0.006433
lag.Mig01    0.00000904177598  0.00000350783189  2.5776              0.009949
TotEmp       0.00897361430701  0.08529282165581  0.1052              0.916210
RemRecv      0.00000000021777  0.00000000013130  1.6585              0.097211
MigPop       0.73604229109122  0.08149504350679  9.0317 < 0.00000000000000022
log(TotPop)  0.32434998374751  0.67385697973250  0.4813              0.630279
               
(Intercept)    
EduExp      ** 
lag.Mig01   ** 
TotEmp         
RemRecv     .  
MigPop      ***
log(TotPop)    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [322]:
generate_summaries(lagMig05)

Call:
rlm(formula = (Score) ~ (EduExp) + (lag.Mig05) + (MigPop) + (TotEmp) + 
    log(TotPop), data = gii_wdi)
Converged in 13 iterations

Coefficients:
   (Intercept)         EduExp      lag.Mig05         MigPop         TotEmp 
18.29061579295  1.63836961240  0.00002656165  0.39701371625 -0.06053234779 
   log(TotPop) 
 0.68525775213 

Degrees of freedom: 132 total; 126 residual
  (16050 observations deleted due to missingness)
Scale estimate: 9.94 

[1] 132

EduExp   lag.Mig05      MigPop      TotEmp log(TotPop) 
   1.150153    1.114905    1.189775    1.031593    1.172086

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,6.9896837,0.0092398102555
lag.Mig05,1,14.7856650,0.0001901811060
MigPop,1,28.5420415,0.0000004141891
TotEmp,1,0.4636633,0.4971663541816
log(TotPop),1,1.3253240,0.2518187051475
Residuals,126,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 31.001, df = 5, p-value = 0.000009364



z test of coefficients:

                 Estimate    Std. Error z value      Pr(>|z|)    
(Intercept) 18.2906157929 11.8046338085  1.5494     0.1212751    
EduExp       1.6383696124  0.6197023216  2.6438     0.0081981 ** 
lag.Mig05    0.0000265617  0.0000069077  3.8452     0.0001204 ***
MigPop       0.3970137163  0.0743126921  5.3425 0.00000009169 ***
TotEmp      -0.0605323478  0.0888968065 -0.6809     0.4959169    
log(TotPop)  0.6852577521  0.5952411234  1.1512     0.2496388    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [323]:
generate_summaries(lagMig10)

Call:
rlm(formula = (Score) ~ (EduExp) + (lag.Mig10) + (MigPop) + (TotEmp) + 
    log(TotPop), data = gii_wdi)
Converged in 11 iterations

Coefficients:
   (Intercept)         EduExp      lag.Mig10         MigPop         TotEmp 
11.20563886040  1.91395052642  0.00001550938  0.43889193808 -0.03824335380 
   log(TotPop) 
 0.94703781240 

Degrees of freedom: 132 total; 126 residual
  (16050 observations deleted due to missingness)
Scale estimate: 9.59 

[1] 132

EduExp   lag.Mig10      MigPop      TotEmp log(TotPop) 
   1.110116    1.131187    1.229982    1.031680    1.154387

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,9.4849575,0.00254365983433
lag.Mig10,1,8.2473364,0.00478969867948
MigPop,1,32.3823739,0.00000008415285
TotEmp,1,0.1776054,0.67415732563416
log(TotPop),1,2.4666644,0.11879233498597
Residuals,126,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 37.345, df = 5, p-value = 0.0000005106



z test of coefficients:

                 Estimate    Std. Error z value      Pr(>|z|)    
(Intercept) 11.2056388604 11.8624265812  0.9446      0.344846    
EduExp       1.9139505264  0.6214597584  3.0798      0.002072 ** 
lag.Mig10    0.0000155094  0.0000054005  2.8718      0.004081 ** 
MigPop       0.4388919381  0.0771264358  5.6906 0.00000001266 ***
TotEmp      -0.0382433538  0.0907460843 -0.4214      0.673439    
log(TotPop)  0.9470378124  0.6029930216  1.5706      0.116284    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [324]:
generate_summaries(lagMig20)

Call:
rlm(formula = (Score) ~ (EduExp) + (lag.Mig20) + (TotEmp) + MigPop + 
    log(TotPop), data = gii_wdi)
Converged in 14 iterations

Coefficients:
    (Intercept)          EduExp       lag.Mig20          TotEmp          MigPop 
13.049866334135  2.126186314785  0.000008941614 -0.043060550603  0.487880226981 
    log(TotPop) 
 0.761901514022 

Degrees of freedom: 132 total; 126 residual
  (16050 observations deleted due to missingness)
Scale estimate: 9.83 

[1] 132

EduExp   lag.Mig20      TotEmp      MigPop log(TotPop) 
   1.101248    1.043116    1.032135    1.145236    1.172361

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,11.2995306,0.001026743393904
lag.Mig20,1,3.1637797,0.077700166514774
TotEmp,1,0.2155323,0.643266477993528
MigPop,1,41.1551070,0.000000002583505
log(TotPop),1,1.5054401,0.222123884068379
Residuals,126,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 33.753, df = 5, p-value = 0.000002666



z test of coefficients:

                 Estimate    Std. Error z value        Pr(>|z|)    
(Intercept) 13.0498663341 12.2043286345  1.0693       0.2849427    
EduExp       2.1261863148  0.6325154188  3.3615       0.0007753 ***
lag.Mig20    0.0000089416  0.0000050270  1.7787       0.0752887 .  
TotEmp      -0.0430605506  0.0927520141 -0.4643       0.6424653    
MigPop       0.4878802270  0.0760503732  6.4152 0.0000000001406 ***
log(TotPop)  0.7619015140  0.6209649589  1.2270       0.2198362    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [325]:
generate_summaries(pat01)

Call:
rlm(formula = log(Pat) ~ (EduExp) + (lag.Mig01) + (TotEmp) + 
    MigPop, data = gii_wdi)
Converged in 9 iterations

Coefficients:
    (Intercept)          EduExp       lag.Mig01          TotEmp          MigPop 
 6.349672188548  0.178142084387  0.000002435176 -0.022346075325 -0.003164038508 

Degrees of freedom: 416 total; 411 residual
  (15766 observations deleted due to missingness)
Scale estimate: 2.54 

[1] 416

EduExp lag.Mig01    TotEmp    MigPop 
 1.046799  1.078361  1.010510  1.042821

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,3.50969833,0.061720011226890
lag.Mig01,1,37.98537815,0.000000001702641
TotEmp,1,2.19517643,0.139209607294923
MigPop,1,0.05826502,0.809379846884286
Residuals,411,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 34.085, df = 4, p-value = 0.000000716



z test of coefficients:

                  Estimate     Std. Error z value         Pr(>|z|)    
(Intercept)  6.34967218855  0.97922516228  6.4844 0.00000000008909 ***
EduExp       0.17814208439  0.09508929665  1.8734          0.06101 .  
lag.Mig01    0.00000243518  0.00000039511  6.1632 0.00000000071277 ***
TotEmp      -0.02234607533  0.01508226417 -1.4816          0.13844    
MigPop      -0.00316403851  0.01310804198 -0.2414          0.80926    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [326]:
generate_summaries(pat05)

Call:
rlm(formula = log(Pat) ~ (EduExp) + (lag.Mig05) + (TotEmp) + 
    MigPop, data = gii_wdi)
Converged in 8 iterations

Coefficients:
    (Intercept)          EduExp       lag.Mig05          TotEmp          MigPop 
 6.146820586424  0.189885292279  0.000002803629 -0.020240235594 -0.002349825124 

Degrees of freedom: 416 total; 411 residual
  (15766 observations deleted due to missingness)
Scale estimate: 2.52 

[1] 416

EduExp lag.Mig05    TotEmp    MigPop 
 1.035443  1.061267  1.012259  1.036278

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,4.1445288,0.042410102657538
lag.Mig05,1,38.3069572,0.000000001464312
TotEmp,1,1.8482688,0.174730277758472
MigPop,1,0.0332466,0.855408558407383
Residuals,411,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 36.085, df = 4, p-value = 0.0000002779



z test of coefficients:

                  Estimate     Std. Error z value        Pr(>|z|)    
(Intercept)  6.14682058642  0.96303446882  6.3828 0.0000000001739 ***
EduExp       0.18988529228  0.09327252755  2.0358         0.04177 *  
lag.Mig05    0.00000280363  0.00000045298  6.1893 0.0000000006045 ***
TotEmp      -0.02024023559  0.01488788475 -1.3595         0.17398    
MigPop      -0.00234982512  0.01288729992 -0.1823         0.85532    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [327]:
generate_summaries(pat10)

Call:
rlm(formula = log(Pat) ~ (EduExp) + (lag.Mig10) + (TotEmp) + 
    MigPop, data = gii_wdi)
Converged in 7 iterations

Coefficients:
    (Intercept)          EduExp       lag.Mig10          TotEmp          MigPop 
 6.142899011473  0.215328817957  0.000002202739 -0.021725801322 -0.002767859893 

Degrees of freedom: 416 total; 411 residual
  (15766 observations deleted due to missingness)
Scale estimate: 2.59 

[1] 416

EduExp lag.Mig10    TotEmp    MigPop 
 1.028319  1.065151  1.012376  1.047454

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,5.13995691,0.023899949723
lag.Mig10,1,21.29990601,0.000005257282
TotEmp,1,2.03938861,0.154030659697
MigPop,1,0.04370893,0.834499666480
Residuals,411,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 31.548, df = 4, p-value = 0.000002367



z test of coefficients:

                  Estimate     Std. Error z value        Pr(>|z|)    
(Intercept)  6.14289901147  0.98384856728  6.2437 0.0000000004272 ***
EduExp       0.21532881796  0.09497786813  2.2671         0.02338 *  
lag.Mig10    0.00000220274  0.00000047728  4.6152 0.0000039275075 ***
TotEmp      -0.02172580132  0.01521338336 -1.4281         0.15327    
MigPop      -0.00276785989  0.01323911692 -0.2091         0.83440    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [328]:
generate_summaries(pat20)

Call:
rlm(formula = log(Pat) ~ (EduExp) + (lag.Mig20) + (TotEmp) + 
    MigPop, data = gii_wdi)
Converged in 6 iterations

Coefficients:
    (Intercept)          EduExp       lag.Mig20          TotEmp          MigPop 
 6.111692807518  0.206102547415  0.000002996138 -0.020532650096 -0.002530704837 

Degrees of freedom: 416 total; 411 residual
  (15766 observations deleted due to missingness)
Scale estimate: 2.59 

[1] 416

EduExp lag.Mig20    TotEmp    MigPop 
 1.020923  1.048927  1.015312  1.036739

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,4.80469478,0.0289425784204
lag.Mig20,1,26.77406095,0.0000003583186
TotEmp,1,1.83988211,0.1757093958385
MigPop,1,0.03739725,0.8467540238358
Residuals,411,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 31.108, df = 4, p-value = 0.00000291



z test of coefficients:

                  Estimate     Std. Error z value        Pr(>|z|)    
(Intercept)  6.11169280752  0.97678007341  6.2570 0.0000000003925 ***
EduExp       0.20610254741  0.09402654048  2.1920         0.02838 *  
lag.Mig20    0.00000299614  0.00000057903  5.1744 0.0000002286865 ***
TotEmp      -0.02053265010  0.01513735539 -1.3564         0.17496    
MigPop      -0.00253070484  0.01308644443 -0.1934         0.84666    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [329]:
generate_summaries(remRecv01)

Call:
rlm(formula = log1p(RemRecv) ~ (EduExp) + (lag.Mig01) + (TotEmp) + 
    MigPop, data = gii_wdi)
Converged in 7 iterations

Coefficients:
    (Intercept)          EduExp       lag.Mig01          TotEmp          MigPop 
24.131312866858 -0.085365560239 -0.000001010371 -0.047708423540 -0.054982121049 

Degrees of freedom: 737 total; 732 residual
  (15445 observations deleted due to missingness)
Scale estimate: 2.42 

[1] 737

EduExp lag.Mig01    TotEmp    MigPop 
 1.057897  1.070725  1.019846  1.041263

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,1.78181,0.1823422629401638406
lag.Mig01,1,56.20850,0.0000000000001887872
TotEmp,1,21.83551,0.0000035355001166643
MigPop,1,28.46017,0.0000001276286670160
Residuals,732,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 25.505, df = 4, p-value = 0.00003982



z test of coefficients:

                  Estimate     Std. Error z value              Pr(>|z|)    
(Intercept) 24.13131286686  0.68295825926 35.3335 < 0.00000000000000022 ***
EduExp      -0.08536556024  0.06395168989 -1.3348                0.1819    
lag.Mig01   -0.00000101037  0.00000013477 -7.4972   0.00000000000006518 ***
TotEmp      -0.04770842354  0.01020971038 -4.6728   0.00000297051774471 ***
MigPop      -0.05498212105  0.01030629910 -5.3348   0.00000009564600602 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [330]:
generate_summaries(remRecv05)

Call:
rlm(formula = log1p(RemRecv) ~ (EduExp) + (lag.Mig05) + (TotEmp) + 
    MigPop, data = gii_wdi)
Converged in 7 iterations

Coefficients:
    (Intercept)          EduExp       lag.Mig05          TotEmp          MigPop 
24.284201714569 -0.092820056315 -0.000001194879 -0.049861553226 -0.055094030651 

Degrees of freedom: 737 total; 732 residual
  (15445 observations deleted due to missingness)
Scale estimate: 2.45 

[1] 737

EduExp lag.Mig05    TotEmp    MigPop 
 1.051733  1.067416  1.021894  1.042093

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,2.134156,0.144478810300233712072
lag.Mig05,1,63.875755,0.000000000000005157519
TotEmp,1,23.974082,0.000001202138158605234
MigPop,1,28.758481,0.000000110043415753655
Residuals,732,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 23.383, df = 4, p-value = 0.0001062



z test of coefficients:

                  Estimate     Std. Error z value              Pr(>|z|)    
(Intercept) 24.28420171457  0.67961861342 35.7321 < 0.00000000000000022 ***
EduExp      -0.09282005632  0.06353729789 -1.4609                 0.144    
lag.Mig05   -0.00000119488  0.00000014951 -7.9922  0.000000000000001325 ***
TotEmp      -0.04986155323  0.01018344707 -4.8963  0.000000976412947130 ***
MigPop      -0.05509403065  0.01027357443 -5.3627  0.000000081990070225 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [331]:
generate_summaries(remRecv10)

Call:
rlm(formula = log1p(RemRecv) ~ (EduExp) + (lag.Mig10) + (TotEmp) + 
    MigPop, data = gii_wdi)
Converged in 6 iterations

Coefficients:
    (Intercept)          EduExp       lag.Mig10          TotEmp          MigPop 
24.293656286249 -0.096834063582 -0.000001232071 -0.049497061607 -0.056429263152 

Degrees of freedom: 737 total; 732 residual
  (15445 observations deleted due to missingness)
Scale estimate: 2.51 

[1] 737

EduExp lag.Mig10    TotEmp    MigPop 
 1.048366  1.062213  1.022503  1.040223

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,2.32772,0.12751985718601815956
lag.Mig10,1,58.68353,0.00000000000005878378
TotEmp,1,23.58576,0.00000146153418700642
MigPop,1,30.19154,0.00000005406190335610
Residuals,732,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 26.75, df = 4, p-value = 0.00002233



z test of coefficients:

                  Estimate     Std. Error z value              Pr(>|z|)    
(Intercept) 24.29365628625  0.67974056905 35.7396 < 0.00000000000000022 ***
EduExp      -0.09683406358  0.06346916987 -1.5257                0.1271    
lag.Mig10   -0.00000123207  0.00000016083 -7.6605   0.00000000000001852 ***
TotEmp      -0.04949706161  0.01019188364 -4.8565   0.00000119468207820 ***
MigPop      -0.05642926315  0.01026979477 -5.4947   0.00000003914136368 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [332]:
generate_summaries(remRecv20)

Call:
rlm(formula = log1p(RemRecv) ~ (EduExp) + (lag.Mig20) + (TotEmp) + 
    MigPop, data = gii_wdi)
Converged in 6 iterations

Coefficients:
    (Intercept)          EduExp       lag.Mig20          TotEmp          MigPop 
24.439521045833 -0.100965409546 -0.000001354609 -0.050758646038 -0.061286900004 

Degrees of freedom: 737 total; 732 residual
  (15445 observations deleted due to missingness)
Scale estimate: 2.54 

[1] 737

EduExp lag.Mig20    TotEmp    MigPop 
 1.050093  1.056053  1.024031  1.030957

,Df,F,Pr(>F)
,<dbl>,<dbl>,<dbl>
EduExp,1,2.400661,0.1217162318664850
lag.Mig20,1,39.440304,0.0000000005797748
TotEmp,1,23.533611,0.0000015004175753
MigPop,1,34.144694,0.0000000076994865
Residuals,732,NA,NA



	studentized Breusch-Pagan test

data:  linear_model
BP = 24.877, df = 4, p-value = 0.00005325



z test of coefficients:

                 Estimate    Std. Error z value              Pr(>|z|)    
(Intercept) 24.4395210458  0.6971351656 35.0571 < 0.00000000000000022 ***
EduExp      -0.1009654095  0.0651639227 -1.5494                0.1213    
lag.Mig20   -0.0000013546  0.0000002157 -6.2802       0.0000000003382 ***
TotEmp      -0.0507586460  0.0104632294 -4.8511       0.0000012275058 ***
MigPop      -0.0612869000  0.0104883228 -5.8433       0.0000000051163 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
